<a href="https://colab.research.google.com/github/SoumyaTCG/Blogging-Agentic-AI/blob/main/Blogging_Agentic_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Importing Required Modules**

In [ ]:
!pip install load_dotenv

In [ ]:
!pip install langchain_groq

In [ ]:
!pip install langgraph

In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client


In [ ]:
!pip install social-post-api

In [ ]:
# Importing required modules
import os  # For accessing environment variables
from dotenv import load_dotenv  # To load environment variables from a .env file
from google.colab import userdata

# Load environment variables from .env file (if present)
#load_dotenv()

# Import ChatGroq from langchain_groq (used for interacting with Groq's LLM API)
from langchain_groq import ChatGroq

# Set API keys for authentication
# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")  # Uncomment if using OpenAI API
os.environ["GROQ_API_KEY"]= userdata.get('GROQ_API_KEY')

# This ensures secure handling of API keys by storing them in environment variables
# instead of hardcoding them directly in the script.

In [ ]:
# Importing required modules
# -----------------------------------------------
# TypedDict: Used for static type checking only
# - No runtime validation, only helps tools like mypy check for type correctness
# - Python itself does NOT enforce these types at runtime
# -----------------------------------------------

# -----------------------------------------------
# BaseModel (from Pydantic): Used for runtime validation
# - Ensures data integrity by validating input values at runtime
# - Converts types if possible (e.g., str to int), else raises an error
# ----------------------------------------------------
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, START, END  # For defining stateful blog_workflows
from typing_extensions import TypedDict  # For defining static type-checked dictionaries
from typing import Optional
from pydantic import BaseModel,Field  # For runtime validation of data
from IPython.display import Image, display
from ayrshare import SocialPost

### **Prompt Chaining**
Prompt chaining decomposes a task into a sequence of steps, where each LLM call processes the output of the previous one. You can add programmatic checks on any intermediate steps to ensure that the process is still on track.

When to use this blog_workflow: This blog_workflow is ideal for situations where the task can be easily and cleanly decomposed into fixed subtasks. The main goal is to trade off latency for higher accuracy, by making each LLM call an easier task.




In [ ]:
# Initialize LLM (Using an open-source model like Llama)
llm=ChatGroq(model="llama-3.3-70b-versatile")

In [ ]:
# Define a single Pydantic model for all stages
class BlogData(BaseModel):
    topic: str
    title: Optional[str] = None
    outline: Optional[str] = None  # Optional because it gets populated step-by-step in the blog_workflow
    draft: Optional[str] = None  # Starts as None and is filled after expansion
    seo_optimized_draft: Optional[str] = None  # SEO optimization happens later
    finalize_blog: Optional[str] = None  # Final blog is generated at the end
    evaluation_score: Optional[float] = None  # Stores evaluation results


In [ ]:
#Define Structured Output Schema
class BlogEvaluation(BaseModel):
    score: int = Field(description="Score the draft between 1-10")
    feedback: str = Field(description="Provide feedback for improvements if score < 7")

In [ ]:
# Augment the LLM with schema for structured output
blog_evaluator = llm.with_structured_output(BlogEvaluation)

In [ ]:
def generate_title(data: BlogData) -> BlogData:
    #Generates a compelling and SEO-friendly blog title.
    print("--GENERATE TITLE--")
    prompt = f"Generate a catchy and SEO-friendly title for a blog about: {data.topic}"
    response = llm([HumanMessage(content=prompt)])
    print(response.content)
    data.title = response.content
    return data

In [ ]:
def generate_outline(data: BlogData) -> BlogData:
    #Generates a blog outline based on the topic.
    print("--GENERATE OUTLINE--")
    prompt = f"Generate a detailed blog outline on the topic: {data.title}"
    response = llm([HumanMessage(content=prompt)])
    print(response.content)
    data.outline = response.content
    return data

In [ ]:
def expand_sections(data: BlogData) -> BlogData:
    #Expands each section of the outline into a full draft.
    print("--EXPAND SECTIONS--")
    sections = data.outline.split('\\n') if data.outline else []
    expanded_content = ""

    for section in sections:
        prompt = f"Write a detailed blog section for: {section}.Try to keep the words less than 200."
        response = llm([HumanMessage(content=prompt)])
        expanded_content += f"\\n{section}\\n{response.content}\\n"
    print(expanded_content)

    data.draft = expanded_content
    return data

In [ ]:
def optimize_for_seo(data: BlogData) -> BlogData:
    #Enhances the blog with SEO optimization.
    print("--OPTIMIZATION FOR SEO--")


    seo_prompt = f"""
    Improve the SEO of the following blog post by incorporating relevant keywords,
    better headings, and meta descriptions.

    BLOG CONTENT:
    {data.draft}

    **Guidelines based on evaluation score:**
    - Score **8.0-10.0**: Minor SEO tweaks only.
    - Score **5.0-7.0**: Enhance structure, improve readability, and add more keywords.
    - Score **<5.0**: Significant rewrite needed to improve clarity, depth, and keyword optimization.

    Current Evaluation Score: {data.evaluation_score}
    """

    print(f"Current Evaluation Score: {data.evaluation_score}")
    optimized_draft = llm.invoke([HumanMessage(content=seo_prompt)]).content

    print(optimized_draft)
    data.seo_optimized_draft = optimized_draft
    return data

In [ ]:
def final_blog(data: BlogData) -> BlogData:
    #Final grammar check and readability improvements.
    print("--GENERATING FINAL CONTENT--")
    prompt = f"Proof read and improve the readability of this blog:\\n{data.seo_optimized_draft}.Give only the final content without anything else."
    response = llm([HumanMessage(content=prompt)])
    print(response.content)
    data.finalize_blog = response.content
    return data

In [ ]:
import re

def evaluate_blog(data: BlogData) -> float:
    print("--EVALUATION IN PROGRESS--")
    """Evaluates the blog’s emotional appeal using sentiment analysis."""

    prompt = f"""
    Analyze the emotional tone of the following blog post.
    Does it feel engaging, persuasive, and impactful?
    Return a sentiment score in float from 1.0 (No sentiment) to 10.0 (Highly engaging).
    Provide only the number, nothing else.

    BLOG CONTENT:
    {data.seo_optimized_draft}
    """

    response = llm.invoke([HumanMessage(content=prompt)]).content

    # Extract float number from response
    match = re.search(r"\d+(\.\d+)?", response)
    if match:
        sentiment_score = float(match.group())  # Convert matched number to float
    else:
        sentiment_score = 5.0  # Default neutral score if parsing fails
    print(sentiment_score)
    data.evaluation_score = sentiment_score
    return data


In [ ]:
def publish_blog_linkedin(data: BlogData):
    print("--PUBLISHING TO LINKEDIN--")
    social = SocialPost('D862BB87-C7F04C16-94716B13-C96C72CB') # get an API Key at ayrshare.com
    # Post to  LinkedIn
    postResult = social.post({'post':data.finalize_blog.str,'platforms': ["linkedin"]})
    print(postResult)


In [ ]:
def publish_blog_reddit(data: BlogData):
    print("--PUBLISHING TO REDDIT--")
    social = SocialPost('D862BB87-C7F04C16-94716B13-C96C72CB') # get an API Key at ayrshare.com
    # Post to  Reddit
    postResult = social.post({'post': 'data.finalize_blog','platforms': ["reddit"]})
    print(postResult)


In [ ]:
def route_based_on_evaluation(data: BlogData):
    if data.evaluation_score < 8.0:
        return "Revise Blog"
    else:
        return "Final Blog"

In [ ]:
# Build Graph
blog_workflow = StateGraph(BlogData)
blog_workflow.add_node("generate_title", generate_title)
blog_workflow.add_node("generate_outline", generate_outline)
blog_workflow.add_node("expand_sections", expand_sections)
blog_workflow.add_node("optimize_for_seo", optimize_for_seo)
blog_workflow.add_node("evaluate_seo_blog", evaluate_blog)
blog_workflow.add_node("final_blog", final_blog)
blog_workflow.add_node("publish_blog_linkedin", publish_blog_linkedin)
#blog_workflow.add_node("publish_blog_reddit", publish_blog_reddit)

# Add Edges
blog_workflow.add_edge(START, "generate_title")
blog_workflow.add_edge("generate_title", "generate_outline")
blog_workflow.add_edge("generate_outline", "expand_sections")
blog_workflow.add_edge("expand_sections", "evaluate_seo_blog")



blog_workflow.add_conditional_edges(
    "evaluate_seo_blog",
    route_based_on_evaluation,
    {
        "Revise Blog": "optimize_for_seo",  # Send back for content improvement
        "Final Blog": "final_blog",  # Send back for final review
    },
)
blog_workflow.add_edge("optimize_for_seo", "evaluate_seo_blog")
blog_workflow.add_edge("final_blog", "publish_blog_linkedin")
#blog_workflow.add_edge("final_blog", "publish_blog_reddit")
blog_workflow.add_edge("publish_blog_linkedin", END)
#blog_workflow.add_edge("publish_blog_reddit", END)

# Compile blog_workflow
blog_generator = blog_workflow.compile()

# Show blog_workflow
display(Image(blog_generator.get_graph().draw_mermaid_png()))

ValueError: Failed to render the graph using the Mermaid.INK API. Status code: 500.

In [ ]:
topic = input("Please enter the topic on which you want to write a post/blog:")
blog_generator.invoke(BlogData(topic=topic))

Please enter the topic on which you want to write a post/blog:What is AI?
--GENERATE TITLE--
Here are a few options:

1. **"Unlocking the Future: What is Artificial Intelligence (AI) and How Does it Work?"**
2. **"The AI Revolution: A Beginner's Guide to Understanding Artificial Intelligence"**
3. **"What is AI? Exploring the Basics and Beyond of Artificial Intelligence Technology"**
4. **"Artificial Intelligence 101: A Comprehensive Guide to AI and its Applications"**
5. **"Demystifying AI: A Beginner's Introduction to Artificial Intelligence and its Potential"**

These titles incorporate relevant keywords (e.g. "Artificial Intelligence", "AI"), and are designed to be informative, attention-grabbing, and optimized for search engines. Choose the one that best fits your blog's tone and style.
--GENERATE OUTLINE--
Based on the provided options, I recommend choosing option 1: **"Unlocking the Future: What is Artificial Intelligence (AI) and How Does it Work?"**. This title effectively cap

{'topic': 'What is AI?',
 'title': 'Here are a few options:\n\n1. **"Unlocking the Future: What is Artificial Intelligence (AI) and How Does it Work?"**\n2. **"The AI Revolution: A Beginner\'s Guide to Understanding Artificial Intelligence"**\n3. **"What is AI? Exploring the Basics and Beyond of Artificial Intelligence Technology"**\n4. **"Artificial Intelligence 101: A Comprehensive Guide to AI and its Applications"**\n5. **"Demystifying AI: A Beginner\'s Introduction to Artificial Intelligence and its Potential"**\n\nThese titles incorporate relevant keywords (e.g. "Artificial Intelligence", "AI"), and are designed to be informative, attention-grabbing, and optimized for search engines. Choose the one that best fits your blog\'s tone and style.',
 'outline': 'Based on the provided options, I recommend choosing option 1: **"Unlocking the Future: What is Artificial Intelligence (AI) and How Does it Work?"**. This title effectively captures the essence of introducing readers to the conc